<a href="https://colab.research.google.com/github/TheG00dB0y/Face_ID/blob/master/Face_Track_Reco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!rm "/content/Face_ID/facepics" -r
!rm "/content/Face_ID/outputs" -r

rm: cannot remove '/content/Face_ID/facepics': No such file or directory


In [1]:
!git clone https://github.com/TheG00dB0y/Face_ID

fatal: destination path 'Face_ID' already exists and is not an empty directory.


In [3]:
%cd Face_ID/

/content/Face_ID


In [3]:
!cp "/content/drive/My Drive/yoloface/cfg" "cfg" -r
!cp "/content/drive/My Drive/yoloface/model-weights" "model-weights" -r
!cp "/content/drive/My Drive/yoloface/samples" "samples" -r
!cp "/content/drive/My Drive/yoloface/utils.py" "utils.py"

In [4]:
!mkdir "facepics"
!mkdir "outputs"

In [5]:
import cv2
import numpy as np
#face_rec
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
from utils import *

Using TensorFlow backend.


In [6]:
def iou(box1, box2):
    """
    Arguments:
    box1 -- first box, list object with coordinates (box1_x1, box1_y1, box1_x2, box_1_y2)
    box2 -- second box, list object with coordinates (box2_x1, box2_y1, box2_x2, box2_y2)
    """
    # Assign variable names to coordinates for clarity
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2
    
    # Calculate the (yi1, xi1, yi2, xi2) coordinates of the intersection of box1 and box2. Calculate its Area.
    ### START CODE HERE ### (≈ 7 lines)
    xi1 = max(box1_x1,box2_x1)
    yi1 = max(box1_y1,box2_y1)
    xi2 = min(box1_x2,box2_x2)
    yi2 = min(box1_y2,box2_y2)
    inter_width = xi2-xi1
    inter_height = yi2-yi1
    inter_area = max(inter_width,0) * max(inter_height,0)
    ### END CODE HERE ###    

    # Calculate the Union area by using Formula: Union(A,B) = A + B - Inter(A,B)
    ### START CODE HERE ### (≈ 3 lines)
    box1_area = (box1_x2-box1_x1) * (box1_y2-box1_y1)
    box2_area = (box2_x2-box2_x1) * (box2_y2-box2_y1)
    union_area = (box1_area + box2_area) - inter_area
    ### END CODE HERE ###
    
    # compute the IoU
    ### START CODE HERE ### (≈ 1 line)
    iou = inter_area / union_area
    ### END CODE HERE ###
    return iou
class Sort(object):
  def __init__(self, max_age=1, iou_thresh =0.4):
    """
    Sets key parameters for SORT
    """
    #Age denotes the expiring of a track
    self.max_age = max_age
    self.iou_thresh = iou_thresh
    self.p_track = []
    self.frame_count = 0
    self.id = 0
    
  def update(self, dets=np.empty((0, 5))):
    # print(dets)
    self.frame_count += 1
    # self.p_track[:]['age'] += 1
    updated_trk = []
    # if self.id==0 and dets.size != 0:
    if not self.p_track and dets.size != 0:
      #Initialise first detections as Previous Trackers & if not first add with previous id(When p_track becomes empty)
      self.p_track = [{'bbox':list(det),'id':k + self.id,'age':0,'frame_count':1} for k,det in enumerate(dets,1)]
      self.id = self.p_track[-1]['id']
    elif self.p_track and dets.size != 0:
      # print("####")
      # print(self.p_track)
      new_p = self.p_track
      for det in dets:
        try:
          best_match_trk = max(new_p,key=lambda x:iou(det,x['bbox']))
        except ValueError:
          print("################################### VALUE ERROR ####################################################\n"+"det: ",end=" ")
          print(det)
          print("prev trk: ",end=" ")
          print(self.p_track)
          continue
        # print(iou(det,best_match_trk['bbox']))
        if iou(det,best_match_trk['bbox']) >= self.iou_thresh :
          best_match_trk['bbox'] = det
          best_match_trk['frame_count'] += 1
          # print(best_match_trk)
          #reset age
          best_match_trk['age'] = 0
          updated_trk.append(best_match_trk)
        else:
          #New non matched detection is assigned to new track
          self.id += 1
          new_trk = {'bbox':det,'id':self.id,'age':0,'frame_count':1}
          updated_trk.append(new_trk)
          if best_match_trk['age'] <= self.max_age:
            best_match_trk['age'] += 1
            updated_trk.append(best_match_trk)
        #removing the used best_match_trk from p_track
        # del dets[dets.index(best_match)]
        # print(self.p_track[self.p_track.index(best_match_trk)])
        # print("before:")
        # print(self.p_track)
        # print(best_match_trk)
        # print(self.p_track.index(best_match_trk))
        # del self.p_track[self.p_track.index(best_match_trk)]
        # self.p_track.remove(best_match_trk)
        self.p_track = list(filter(lambda x: x['id'] != best_match_trk['id'] and x['age'] < self.max_age, self.p_track)) 
        # print("after:")
        # print(self.p_track)        
      #increase age of previous tracks
      for trk in self.p_track:
        trk['age'] += 1
      self.p_track = self.p_track + updated_trk
    elif self.p_track and dets.size == 0:
      new_p = []
      for trk in self.p_track:
        trk['age'] += 1
        if(trk['age'] < self.max_age):
          new_p.append(trk)
      self.p_track = new_p
      return []
        
      
    # print("#######",end = " ")
    # print(self.p_track,end="#######")
    return self.p_track




In [7]:
model_cfg = './cfg/yolov3-face.cfg'
model_weights = './model-weights/yolov3-wider_16000.weights'
# image = './samples/outside_000001.jpg'
video = './samples/sss.mp4'
# output_dir = '/content/drive/My Drive/yoloface/outputs/'
output_dir = './outputs/'

IOU_THRESH = 0.35
MAX_AGE = 3
#Trackers(Here faces) with given frame count value is chosen for age and gender processing
FRAME_COUNT_THRESH = 6
# DETECT_INTERVAL = 2
FACE_HEIGHT = 50
FACE_WIDTH = 50
# print the arguments
print('----- info -----')
print('[i] The config file: ',model_cfg)
print('[i] The weights of model file: ',model_weights)
# print('[i] Path to image file: ',image)
print('[i] Path to video file: ',video)
print('[i] IOU Threshold value: ',IOU_THRESH)
print('[i] Max age value: ',MAX_AGE)
print('[i] Frame count threshold value: ',FRAME_COUNT_THRESH)
# print('[i] Detect interval: ',DETECT_INTERVAL)
print('[i] Minimum face height required: ',FACE_HEIGHT)
print('[i] Minimum face width required: ',FACE_WIDTH)


#Read config net
net = cv2.dnn.readNetFromDarknet(model_cfg,model_weights)
print('###########################################################\n')


#VIDEO
cap = cv2.VideoCapture(video)
output_file = video[:-4].rsplit('/')[-1] + '_keras.avi'
output_file = output_dir + output_file

# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

video_writer = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc(*'XVID'),cap.get(cv2.CAP_PROP_FPS), (
                                          round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                                          round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))


# print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(cap.get(cv2.CAP_PROP_FPS))
#create instance of SORT
mot = Sort(MAX_AGE,IOU_THRESH)

----- info -----
[i] The config file:  ./cfg/yolov3-face.cfg
[i] The weights of model file:  ./model-weights/yolov3-wider_16000.weights
[i] Path to video file:  ./samples/sss.mp4
[i] IOU Threshold value:  0.35
[i] Max age value:  3
[i] Frame count threshold value:  6
[i] Minimum face height required:  50
[i] Minimum face width required:  50
###########################################################

23.976023976023978


Face Recognisation Model

**BUG**:
**In line 506 I changed line**

_LOCAL_DEVICES = tf.config.experimental_list_devices()

to

devices = tf.config.list_logical_devices()

_LOCAL_DEVICES = [x.name for x in devices]

**And restart runtime**

In [8]:
# Create a new model instance
from keras import backend as K
K.set_image_data_format('channels_first')
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [9]:
# Restore the weights
FRmodel.load_weights('./weight_file')

In [10]:
def img_encoding(img1, model):
    img = img1[...,::-1]
    img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

In [11]:
def who_is_it(img, database, model):
        
    encoding = img_encoding(img,model)
    min_dist = 100
    for (name, db_enc) in database.items():
      # Compute L2 distance between the target "encoding" and the current db_enc from the database. (≈ 1 line)
      dist = np.linalg.norm(db_enc-encoding)
      # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
      if dist<min_dist:
        min_dist = dist
        identity = name

    if min_dist > 0.7:
        # print("Not in the database.")
        return ''
    else:
        # print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        return identity
        
    return identity

In [10]:
# !rm /content/Face_ID/images/facepics -r

In [11]:
# !unzip "/content/file(1).zip" -d /content/Face_ID/images

In [12]:
database = {}
database["Athul"] = img_to_encoding("/content/Face_ID/images/content/age-gender-estimation/facepics/1.jpg", FRmodel)
database["Alveena"] = img_to_encoding("/content/Face_ID/images/content/age-gender-estimation/facepics/2.jpg", FRmodel)
database["Ajay"] = img_to_encoding("/content/Face_ID/images/content/age-gender-estimation/facepics/3.jpg", FRmodel)

In [13]:
# who_is_it("/content/Face_ID/images/content/age-gender-estimation/facepics/1.jpg", database, FRmodel)

In [14]:
def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=1.0, thickness=3):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)

In [15]:
margin = 0.4

In [17]:
#Frame counter for detect interval
# c=0
while True:
  
  has_frame, frame = cap.read()
  if not has_frame:
      print('[i] ==> Done processing!!!')
      #print('[i] ==> Output file is stored at', os.path.join(args.output_dir, output_file))
      break
  # frame = cv2.resize(frame, (0, 0), fx=0.7, fy=0.7)
  # Resize, Convert BGR to HSV
  # if ((IMG_HEIGHT, IMG_WIDTH) != frame.shape[0:2]):
  #     frame = cv2.resize(frame, dsize=(IMG_WIDTH, IMG_HEIGHT), fx=0, fy=0)
  # else:
  #     frame = frame
  # if(c % DETECT_INTERVAL==0):
  new_frame = frame.copy()
  img_h, img_w, _ = np.shape(frame)

  blob = cv2.dnn.blobFromImage(frame, 1 / 255, (IMG_WIDTH, IMG_HEIGHT),[0, 0, 0], 1, crop=False)
  net.setInput(blob)
  # Runs the forward pass to get output of the output layers
  outs = net.forward(get_outputs_names(net))
  # Remove the bounding boxes with low confidence
  faces = post_process(frame, outs,CONF_THRESHOLD, NMS_THRESHOLD)
  faces = np.array(faces) 
  
  # faces_resized = []

  if(faces.shape[0]>0):
    final_faces = faces[:,:4]
    
  else:
    final_faces = faces
  trackers = mot.update(final_faces)
  # print("Final faces:",end=" ")
  # print(final_faces)
  # print("$$$$$$$$$$$$$ ") 
  # print(trackers)
  # print("&&&&&&&&&&&&&&&&&&")
  print("#",end="")
  # faces_resized = np.empty((len(trackers), img_size, img_size, 3))

  for i,trk in enumerate(trackers):
    box = np.array(trk['bbox'])
    id = int(trk['id'])
    frame_count = trk['frame_count']
    d = box.astype(np.int32)
    (startX, startY, endX, endY) = (d[0],d[1],d[2],d[3])
    face_h = endY - startY
    face_w = endX - startX

    x1, y1, x2, y2, w, h = d[0], d[1], d[2] + 1, d[3] + 1, face_w, face_h
    xw1 = max(int(x1 - margin * w), 0)
    yw1 = max(int(y1 - margin * h), 0)
    xw2 = min(int(x2 + margin * w), img_w - 1)
    yw2 = min(int(y2 + margin * h), img_h - 1)
    # cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
    cv2.rectangle(frame, (xw1, yw1), (xw2, yw2), (255, 0, 0), 2)
    # faces[i, :, :, :] = cv2.resize(img[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))




    # if(frame_count==FRAME_COUNT_THRESH and (face_h > FACE_HEIGHT and face_w > FACE_WIDTH)):
    # crop_face = new_frame[startY:endY, startX:endX]
    # crop_face_shape = new_frame[yw1:yw2 + 1, xw1:xw2 + 1, :]
    crop_face = cv2.resize(new_frame[yw1:yw2 + 1, xw1:xw2 + 1, :], (96, 96))
    # crop_face = cv2.resize( new_frame[startY:endY, startX:endX], (96, 96))
    print(crop_face.shape)
    # faces_resized[i,:,:,:] = crop_face

    if(frame_count == FRAME_COUNT_THRESH):  
      try:
        cv2.imwrite("{0}/{1}.jpg".format("facepics",id),crop_face)
        # cv2_imshow(crop_face)
      except:
        print("#########################Image error####################")
        continue
    identity = who_is_it(crop_face, database, FRmodel)    
    # print(identity)
    label = "{}, {}".format(id,identity)
    draw_label(frame, (d[0], d[1]), label)





  # cv2_imshow(frame)
  video_writer.write(frame.astype(np.uint8))
  # c+=1



KeyboardInterrupt: ignored

In [18]:
cap.release()
video_writer.release()